In [5]:
from sklearn import preprocessing
from time import time
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import csv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_diabetes
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import silhouette_samples, silhouette_score
from operator import truediv
from sklearn.metrics import pairwise_distances
import pandas as pd
from sklearn.feature_selection import VarianceThreshold

get_ipython().magic('matplotlib')

df=pd.read_csv('C:/diabetic_data_processed_withweight.csv',';')
df=df.reset_index()
df=df.groupby(['patient_nbr'], sort=True).last()
to_del = ['encounter_id', 'patient_nbr','medical_specialty','payer_code','index','admission_type_id','discharge_disposition_id','admission_source_id']
print (to_del)
#Filter_selected cols
filtered_cols = [c for c in df.columns if (c not in to_del) ]#and ('ENF' not in c)
df_2 = df[filtered_cols]
print ("df_2",df_2.shape)
print(df_2.columns)

# Filter complete null columns
cols = np.where((np.sum(df_2.isnull(), axis=0).values) == df_2.shape[0])[0]
print (cols)
filt_cols = [c for c in df_2.columns if c not in df_2.columns[cols]]
df_3 = df_2[filt_cols]
print ("df_3",df_3.shape)

#Fill na
df_4 = df_3.fillna(value=np.mean(df_3,axis=0),inplace=False,axis=0).values
print ("df_4",df_4.shape)
data=df_4

selector = VarianceThreshold(threshold=(.99 * (1 - .99)))
newdata=selector.fit_transform(data)
idxs = selector.get_support(indices=True)
print(data[:, idxs])
print("indices",idxs)
columnslist=df_2.columns.tolist()
print("lenindex",len(idxs))
for z in range(0,len(columnslist)):
    if z not in idxs:
        print(columnslist[z])
print("after",newdata.shape)
print("initial",data.shape)
print("Headers_FINAL: ", df_2.columns.values.tolist())
df_2=df_2.reset_index()

Using matplotlib backend: Qt5Agg
['encounter_id', 'patient_nbr', 'medical_specialty', 'payer_code', 'index', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id']
df_2 (71518, 43)
Index(['race', 'gender', 'age', 'weight', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')
[]
df_3 (71518, 43)
df_

In [6]:
for i in range (0,df_2['diag_3'].size):
    if ((df_2.loc[i,'diag_1']==410 or df_2.loc[i,'diag_1']==412) or (df_2.loc[i,'diag_2']==410 or df_2.loc[i,'diag_2']==412) or (df_2.loc[i,'diag_3']==410 or df_2.loc[i,'diag_3']==412)):
        #print(df_2['diag_1'],df_2['diag_2'],df_2['diag_3'])
        df_2.loc[i,'Myocardial_infarction']=1
    else:
        df_2.loc[i,'Myocardial_infarction']=0

In [7]:
df_2w = df_2[df_2['weight']>0]
df_2a = df_2[df_2['A1Cresult']>0]
print("df_2w: ", df_2w.shape)
print("df_2a: ", df_2a.shape)

df_2w:  (2866, 45)
df_2a:  (12389, 45)


In [10]:
print(df_2['Myocardial_infarction'].astype('category').value_counts())
listA=df_2w[df_2w['Myocardial_infarction']==1]
listC=df_2w[df_2w['Myocardial_infarction']==0] 
print(df_2w['weight'].describe())
mean=df_2w['weight'].mean()
print("mean",mean)
listB=df_2w[df_2w['weight']>mean]
listD=df_2w[df_2w['weight']<mean]
#No healty and bad social
interA=listB[listB.isin(listA)]
interA=interA[interA['weight']>0]
a=len(interA)
#Healthy and bad social
interB=listB[listB.isin(listC)]
interB=interB[interB['weight']>0]
b=len(interB)
#No healty and social
interC=listD[listD.isin(listA)]
interC=interC[interC['weight']>0]
c=len(interC)
#Healthy and social
interD=listD[listD.isin(listC)]
interD=interD[interD['weight']>0]
d=len(interD)

print(len(listB), len(listA), len(listC),len(listD))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","RR: ",RR,", RD: ",RD,", OR: ",OR)

print(df_2a['A1Cresult'].describe())
print("mean",mean)

listA=df_2a[df_2a['Myocardial_infarction']==1]
listC=df_2a[df_2a['Myocardial_infarction']==0] 
print(df_2a['A1Cresult'].describe())
mean=df_2a['A1Cresult'].mean()
print("mean",mean)
listB=df_2a[df_2a['A1Cresult']>6.5]
listD=df_2a[df_2a['A1Cresult']<6.5]
#No healty and bad social
interA=listB[listB.isin(listA)]
interA=interA[interA['A1Cresult']>0]
a=len(interA)
#Healthy and bad social
interB=listB[listB.isin(listC)]
interB=interB[interB['A1Cresult']>0]
b=len(interB)
#No healty and social
interC=listD[listD.isin(listA)]
interC=interC[interC['A1Cresult']>0]
c=len(interC)
#Healthy and social
interD=listD[listD.isin(listC)]
interD=interD[interD['A1Cresult']>0]
d=len(interD)

print(len(listB), len(listA), len(listC),len(listD))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("A1Cresult-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listMI=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Myocardial_infarction']==1):
        listMI.append(df_2.loc[x,:])    
df_MI = pd.DataFrame(listMI)
print("Description age: ", df_MI['age'].describe())
df_MI['age'].describe()

import scipy as sp
import scipy.stats
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0*np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h
print("Age",mean_confidence_interval(df_MI['age']))
print("Gender",df_MI['gender'].astype('category').value_counts())
print("Race",df_MI['race'].astype('category').value_counts())
print("Readmitted",df_MI['readmitted'].astype('category').value_counts())

0.0    67906
1.0     3612
Name: Myocardial_infarction, dtype: int64
count    2866.000000
mean       93.315073
std        25.400488
min        25.000000
25%        75.000000
50%        88.000000
75%       125.000000
max       201.000000
Name: weight, dtype: float64
mean 93.31507327285415
760 148 2718 2106
37 723 111 1995
Weight--> RR:  0.9236842105263158 , RD:  -0.004022342180236921 , OR:  0.3333333333333333
count    12389.000000
mean         6.357172
std          3.549122
min          1.000000
25%          1.000000
50%          8.000000
75%          9.000000
max          9.000000
Name: A1Cresult, dtype: float64
mean 93.31507327285415
count    12389.000000
mean         6.357172
std          3.549122
min          1.000000
25%          1.000000
50%          8.000000
75%          9.000000
max          9.000000
Name: A1Cresult, dtype: float64
mean 6.357171684558883
8642 801 11588 3747
600 8042 201 3546
A1Cresult--> RR:  1.2942692231966757 , RD:  0.01578545873032608 , OR:  2.985074626865672


In [11]:
for i in range (0,df_2['diag_3'].size):
    if (df_2.loc[i,'diag_1']==428 or df_2.loc[i,'diag_2']==428 or (df_2.loc[i,'diag_3']==428)):
        #print(df_2['diag_1'],df_2['diag_2'],df_2['diag_3'])
        df_2.loc[i,'Congestive_heart_failure']=1
    else:
        df_2.loc[i,'Congestive_heart_failure']=0      

In [12]:
df_2w = df_2[df_2['weight']>0]
df_2a = df_2[df_2['A1Cresult']>0]
print("df_2w: ", df_2w.shape)
print("df_2a: ", df_2a.shape)

df_2w:  (2866, 46)
df_2a:  (12389, 46)


In [14]:
print(df_2['Congestive_heart_failure'].astype('category').value_counts())
listA=df_2w[df_2w['Congestive_heart_failure']==1]
listC=df_2w[df_2w['Congestive_heart_failure']==0] 
print(df_2w['weight'].describe())
mean=df_2w['weight'].mean()
print("mean",mean)
listB=df_2w[df_2w['weight']>mean]
listD=df_2w[df_2w['weight']<mean]
#No healty and bad social
interA=listB[listB.isin(listA)]
interA=interA[interA['weight']>0]
a=len(interA)
#Healthy and bad social
interB=listB[listB.isin(listC)]
interB=interB[interB['weight']>0]
b=len(interB)
#No healty and social
interC=listD[listD.isin(listA)]
interC=interC[interC['weight']>0]
c=len(interC)
#Healthy and social
interD=listD[listD.isin(listC)]
interD=interD[interD['weight']>0]
d=len(interD)

print(len(listB), len(listA), len(listC),len(listD))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","RR: ",RR,", RD: ",RD,", OR: ",OR)

listA=df_2a[df_2a['Congestive_heart_failure']==1]
listC=df_2a[df_2a['Congestive_heart_failure']==0] 
print(df_2a['A1Cresult'].describe())
mean=df_2a['A1Cresult'].mean()
print("mean",mean)
listB=df_2a[df_2a['A1Cresult']>6.5]
listD=df_2a[df_2a['A1Cresult']<6.5]
#No healty and bad social
interA=listB[listB.isin(listA)]
interA=interA[interA['A1Cresult']>0]
a=len(interA)
#Healthy and bad social
interB=listB[listB.isin(listC)]
interB=interB[interB['A1Cresult']>0]
b=len(interB)
#No healty and social
interC=listD[listD.isin(listA)]
interC=interC[interC['A1Cresult']>0]
c=len(interC)
#Healthy and social
interD=listD[listD.isin(listC)]
interD=interD[interD['A1Cresult']>0]
d=len(interD)

print(len(listB), len(listA), len(listC),len(listD))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("A1CResult-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listCHF=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Congestive_heart_failure']==1):
        listCHF.append(df_2.loc[x,:])    
df_CHF = pd.DataFrame(listCHF)
print("Age: ", df_CHF['age'].describe())

print("Age",mean_confidence_interval(df_CHF['age']))
print("Gender",df_CHF['gender'].astype('category').value_counts())
print("Race",df_CHF['race'].astype('category').value_counts())
print("Readmitted",df_CHF['readmitted'].astype('category').value_counts())

0.0    60527
1.0    10991
Name: Congestive_heart_failure, dtype: int64
count    2866.000000
mean       93.315073
std        25.400488
min        25.000000
25%        75.000000
50%        88.000000
75%       125.000000
max       201.000000
Name: weight, dtype: float64
mean 93.31507327285415
760 393 2473 2106
97 663 296 1810
Weight--> RR:  0.9080814366998579 , RD:  -0.012919228270105443 , OR:  0.3277027027027027
count    12389.000000
mean         6.357172
std          3.549122
min          1.000000
25%          1.000000
50%          8.000000
75%          9.000000
max          9.000000
Name: A1Cresult, dtype: float64
mean 6.357171684558883
8642 1795 10594 3747
1193 7449 602 3145
A1CResult--> RR:  0.8592378179346635 , RD:  -0.022615114385730606 , OR:  1.9817275747508305
Age:  count    10991.000000
mean        77.578018
std         12.587553
min         20.000000
25%         70.000000
50%         80.000000
75%         90.000000
max        100.000000
Name: age, dtype: float64
Age (77.5780183

In [15]:
for i in range (0,df_2['diag_3'].size):
    if (df_2.loc[i,'diag_1'] in (443,441,785,43) 
        or df_2.loc[i,'diag_2'] in (443,441,785,43)  
        or df_2.loc[i,'diag_3'] in (443,441,785,43)):
        df_2.loc[i,'Peripheral_vascular_disease']=1
    else:
        df_2.loc[i,'Peripheral_vascular_disease']=0

In [16]:
df_2w = df_2[df_2['weight']>0]
df_2a = df_2[df_2['A1Cresult']>0]
print("df_2w: ", df_2w.shape)
print("df_2a: ", df_2a.shape)

df_2w:  (2866, 47)
df_2a:  (12389, 47)


In [17]:
print(df_2['Peripheral_vascular_disease'].astype('category').value_counts())
listA=df_2w[df_2w['Peripheral_vascular_disease']==1]
listC=df_2w[df_2w['Peripheral_vascular_disease']==0] 
print(df_2w['weight'].describe())
mean=df_2w['weight'].mean()
print("mean",mean)
listB=df_2w[df_2w['weight']>mean]
listD=df_2w[df_2w['weight']<mean]
#No healty and bad social
interA=listB[listB.isin(listA)]
interA=interA[interA['weight']>0]
a=len(interA)
#Healthy and bad social
interB=listB[listB.isin(listC)]
interB=interB[interB['weight']>0]
b=len(interB)
#No healty and social
interC=listD[listD.isin(listA)]
interC=interC[interC['weight']>0]
c=len(interC)
#Healthy and social
interD=listD[listD.isin(listC)]
interD=interD[interD['weight']>0]
d=len(interD)

print(len(listB), len(listA), len(listC),len(listD))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","RR: ",RR,", RD: ",RD,", OR: ",OR)

listA=df_2a[df_2a['Peripheral_vascular_disease']==1]
listC=df_2a[df_2a['Peripheral_vascular_disease']==0] 
print(df_2a['A1Cresult'].describe())
mean=df_2a['A1Cresult'].mean()
print("mean",mean)
listB=df_2a[df_2a['A1Cresult']>6.5]
listD=df_2a[df_2a['A1Cresult']<6.5]
#No healty and bad social
interA=listB[listB.isin(listA)]
interA=interA[interA['A1Cresult']>0]
a=len(interA)
#Healthy and bad social
interB=listB[listB.isin(listC)]
interB=interB[interB['A1Cresult']>0]
b=len(interB)
#No healty and social
interC=listD[listD.isin(listA)]
interC=interC[interC['A1Cresult']>0]
c=len(interC)
#Healthy and social
interD=listD[listD.isin(listC)]
interD=interD[interD['A1Cresult']>0]
d=len(interD)

print(len(listB), len(listA), len(listC),len(listD))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("A1CResult-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listCHF=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Peripheral_vascular_disease']==1):
        listCHF.append(df_2.loc[x,:])    
df_CHF = pd.DataFrame(listCHF)
print(df_CHF['age'].describe())

print("Age",mean_confidence_interval(df_CHF['age']))
print("Gender",df_CHF['gender'].astype('category').value_counts())
print("Race",df_CHF['race'].astype('category').value_counts())
print("Readmitted",df_CHF['readmitted'].astype('category').value_counts())

0.0    70170
1.0     1348
Name: Peripheral_vascular_disease, dtype: int64
count    2866.000000
mean       93.315073
std        25.400488
min        25.000000
25%        75.000000
50%        88.000000
75%       125.000000
max       201.000000
Name: weight, dtype: float64
mean 93.31507327285415
760 62 2804 2106
23 737 39 2067
Weight--> RR:  1.6342105263157896 , RD:  0.011744639376218325 , OR:  0.5897435897435898
count    12389.000000
mean         6.357172
std          3.549122
min          1.000000
25%          1.000000
50%          8.000000
75%          9.000000
max          9.000000
Name: A1Cresult, dtype: float64
mean 6.357171684558883
8642 208 12181 3747
142 8500 66 3681
A1CResult--> RR:  0.9328543476888768 , RD:  -0.0011827096483944845 , OR:  2.1515151515151514
count    1348.00000
mean       72.45549
std        13.76363
min        10.00000
25%        60.00000
50%        70.00000
75%        80.00000
max       100.00000
Name: age, dtype: float64
Age (72.455489614243319, 71.72008466792

In [21]:
import scipy.stats as stats
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0*np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h

In [18]:
for i in range (0,df_2['diag_3'].size):
    if ((df_2.loc[i,'diag_1']>=430 and df_2.loc[i,'diag_1']<=438) or (df_2.loc[i,'diag_2']>=430 and df_2.loc[i,'diag_2']<=438) or (df_2.loc[i,'diag_3']>=430 and df_2.loc[i,'diag_3']<=438)):
        df_2.loc[i,'Cerebrovascular_disease']=1
    else:
        df_2.loc[i,'Cerebrovascular_disease']=0

In [19]:
df_2w = df_2[df_2['weight']>0]
df_2a = df_2[df_2['A1Cresult']>0]
print("df_2w: ", df_2w.shape)
print("df_2a: ", df_2a.shape)

df_2w:  (2866, 48)
df_2a:  (12389, 48)


In [20]:
print(df_2['Cerebrovascular_disease'].astype('category').value_counts())
listA=df_2w[df_2w['Cerebrovascular_disease']==1]
listC=df_2w[df_2w['Cerebrovascular_disease']==0] 
print(df_2w['weight'].describe())
mean=df_2w['weight'].mean()
print("mean",mean)
listB=df_2w[df_2w['weight']>mean]
listD=df_2w[df_2w['weight']<mean]
#No healty and bad social
interA=listB[listB.isin(listA)]
interA=interA[interA['weight']>0]
a=len(interA)
#Healthy and bad social
interB=listB[listB.isin(listC)]
interB=interB[interB['weight']>0]
b=len(interB)
#No healty and social
interC=listD[listD.isin(listA)]
interC=interC[interC['weight']>0]
c=len(interC)
#Healthy and social
interD=listD[listD.isin(listC)]
interD=interD[interD['weight']>0]
d=len(interD)

print(len(listB), len(listA), len(listC),len(listD))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","RR: ",RR,", RD: ",RD,", OR: ",OR)

listA=df_2a[df_2a['Cerebrovascular_disease']==1]
listC=df_2a[df_2a['Cerebrovascular_disease']==0] 
print(df_2a['A1Cresult'].describe())
mean=df_2a['A1Cresult'].mean()
print("mean",mean)
listB=df_2a[df_2a['A1Cresult']>6.5]
listD=df_2a[df_2a['A1Cresult']<6.5]
#No healty and bad social
interA=listB[listB.isin(listA)]
interA=interA[interA['A1Cresult']>0]
a=len(interA)
#Healthy and bad social
interB=listB[listB.isin(listC)]
interB=interB[interB['A1Cresult']>0]
b=len(interB)
#No healty and social
interC=listD[listD.isin(listA)]
interC=interC[interC['A1Cresult']>0]
c=len(interC)
#Healthy and social
interD=listD[listD.isin(listC)]
interD=interD[interD['A1Cresult']>0]
d=len(interD)

print(len(listB), len(listA), len(listC),len(listD))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("A1CResult-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listCHF=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Cerebrovascular_disease']==1):
        listCHF.append(df_2.loc[x,:])    
df_CHF = pd.DataFrame(listCHF)
print(df_CHF['age'].describe())

print("Age",mean_confidence_interval(df_CHF['age']))
print("Gender",df_CHF['gender'].astype('category').value_counts())
print("Race",df_CHF['race'].astype('category').value_counts())
print("Readmitted",df_CHF['readmitted'].astype('category').value_counts())

0.0    67453
1.0     4065
Name: Cerebrovascular_disease, dtype: int64
count    2866.000000
mean       93.315073
std        25.400488
min        25.000000
25%        75.000000
50%        88.000000
75%       125.000000
max       201.000000
Name: weight, dtype: float64
mean 93.31507327285415
760 138 2728 2106
27 733 111 1995
Weight--> RR:  0.6740398293029871 , RD:  -0.017180236917079025 , OR:  0.24324324324324326
count    12389.000000
mean         6.357172
std          3.549122
min          1.000000
25%          1.000000
50%          8.000000
75%          9.000000
max          9.000000
Name: A1Cresult, dtype: float64
mean 6.357171684558883
8642 823 11566 3747
575 8067 248 3499
A1CResult--> RR:  1.0052766496704018 , RD:  0.00034924182499590506 , OR:  2.318548387096774
count    4065.000000
mean       75.857319
std        12.335904
min        10.000000
25%        70.000000
50%        80.000000
75%        80.000000
max       100.000000
Name: age, dtype: float64
Age (75.857318573185736, 75.477

In [21]:
for i in range (0,df_2['diag_3'].size):
    if (df_2.loc[i,'diag_1']==290 or df_2.loc[i,'diag_2']==290 or df_2.loc[i,'diag_3']==290):
        df_2.loc[i,'Dementia']=1
    else:
        df_2.loc[i,'Dementia']=0

In [22]:
df_2w = df_2[df_2['weight']>0]
df_2a = df_2[df_2['A1Cresult']>0]
print("df_2w: ", df_2w.shape)
print("df_2a: ", df_2a.shape)

df_2w:  (2866, 49)
df_2a:  (12389, 49)


In [23]:
import scipy.stats as stats

In [24]:
print(df_2['Dementia'].astype('category').value_counts())
listA=df_2w[df_2w['Dementia']==1]
listC=df_2w[df_2w['Dementia']==0] 
print(df_2w['weight'].describe())
mean=df_2w['weight'].mean()
print("mean",mean)
listB=df_2w[df_2w['weight']>mean]
listD=df_2w[df_2w['weight']<mean]
#No healty and bad social
interA=listB[listB.isin(listA)]
interA=interA[interA['weight']>0]
a=len(interA)
#Healthy and bad social
interB=listB[listB.isin(listC)]
interB=interB[interB['weight']>0]
b=len(interB)
#No healty and social
interC=listD[listD.isin(listA)]
interC=interC[interC['weight']>0]
c=len(interC)
#Healthy and social
interD=listD[listD.isin(listC)]
interD=interD[interD['weight']>0]
d=len(interD)

print(len(listB), len(listA), len(listC),len(listD))
print(a,b,c,d)

OR=(a*d)/(c*d)
#RR=(a/(a+b)) / (c/(c+d))
#RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","RR: ",RR,", RD: ",RD,", OR: ",OR)

listA=df_2a[df_2a['Dementia']==1]
listC=df_2a[df_2a['Dementia']==0] 
print(df_2a['A1Cresult'].describe())
mean=df_2a['A1Cresult'].mean()
print("mean",mean)
listB=df_2a[df_2a['A1Cresult']>6.5]
listD=df_2a[df_2a['A1Cresult']<6.5]
#No healty and bad social
interA=listB[listB.isin(listA)]
interA=interA[interA['A1Cresult']>0]
a=len(interA)
#Healthy and bad social
interB=listB[listB.isin(listC)]
interB=interB[interB['A1Cresult']>0]
b=len(interB)
#No healty and social
interC=listD[listD.isin(listA)]
interC=interC[interC['A1Cresult']>0]
c=len(interC)
#Healthy and social
interD=listD[listD.isin(listC)]
interD=interD[interD['A1Cresult']>0]
d=len(interD)

print(len(listB), len(listA), len(listC),len(listD))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("A1CResult-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listCHF=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Dementia']==1):
        listCHF.append(df_2.loc[x,:])    
df_CHF = pd.DataFrame(listCHF)
print(df_CHF['age'].describe())

print("Age",mean_confidence_interval(df_CHF['age']))
print("Gender",df_CHF['gender'].astype('category').value_counts())
print("Race",df_CHF['race'].astype('category').value_counts())
print("Readmitted",df_CHF['readmitted'].astype('category').value_counts())

0.0    71352
1.0      166
Name: Dementia, dtype: int64
count    2866.000000
mean       93.315073
std        25.400488
min        25.000000
25%        75.000000
50%        88.000000
75%       125.000000
max       201.000000
Name: weight, dtype: float64
mean 93.31507327285415
760 3 2863 2106
0 760 3 2103
Weight--> RR:  1.0052766496704018 , RD:  0.00034924182499590506 , OR:  0.0
count    12389.000000
mean         6.357172
std          3.549122
min          1.000000
25%          1.000000
50%          8.000000
75%          9.000000
max          9.000000
Name: A1Cresult, dtype: float64
mean 6.357171684558883
8642 30 12359 3747
18 8624 12 3735
A1CResult--> RR:  0.6503702846563296 , RD:  -0.001119710857785974 , OR:  1.5
count    166.000000
mean      83.554217
std        9.724387
min       50.000000
25%       80.000000
50%       90.000000
75%       90.000000
max      100.000000
Name: age, dtype: float64
Age (83.554216867469876, 82.06398658277493, 85.044447152164821)
Gender 1.0    94
0.0    72
N

In [25]:
for i in range (0,df_2['diag_3'].size):
    if ((df_2.loc[i,'diag_1']>=490 and df_2.loc[i,'diag_1']<=506) or (df_2.loc[i,'diag_2']>=490 and df_2.loc[i,'diag_2']<=506) or (df_2.loc[i,'diag_3']>=490 and df_2.loc[i,'diag_3']<=506)):
        df_2.loc[i,'Chronic_pulmonary_disease']=1
    else:
        df_2.loc[i,'Chronic_pulmonary_disease']=0

In [26]:
df_2w = df_2[df_2['weight']>0]
df_2a = df_2[df_2['A1Cresult']>0]
print("df_2w: ", df_2w.shape)
print("df_2a: ", df_2a.shape)

df_2w:  (2866, 50)
df_2a:  (12389, 50)


In [27]:
print(df_2['Chronic_pulmonary_disease'].astype('category').value_counts())
listA=df_2w[df_2w['Chronic_pulmonary_disease']==1]
listC=df_2w[df_2w['Chronic_pulmonary_disease']==0] 
print(df_2w['weight'].describe())
mean=df_2w['weight'].mean()
print("mean",mean)
listB=df_2w[df_2w['weight']>mean]
listD=df_2w[df_2w['weight']<mean]
#No healty and bad social
interA=listB[listB.isin(listA)]
interA=interA[interA['weight']>0]
a=len(interA)
#Healthy and bad social
interB=listB[listB.isin(listC)]
interB=interB[interB['weight']>0]
b=len(interB)
#No healty and social
interC=listD[listD.isin(listA)]
interC=interC[interC['weight']>0]
c=len(interC)
#Healthy and social
interD=listD[listD.isin(listC)]
interD=interD[interD['weight']>0]
d=len(interD)

print(len(listB), len(listA), len(listC),len(listD))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("Weight-->","RR: ",RR,", RD: ",RD,", OR: ",OR)

listA=df_2a[df_2a['Chronic_pulmonary_disease']==1]
listC=df_2a[df_2a['Chronic_pulmonary_disease']==0] 
print(df_2a['A1Cresult'].describe())
mean=df_2a['A1Cresult'].mean()
print("mean",mean)
listB=df_2a[df_2a['A1Cresult']>6.5]
listD=df_2a[df_2a['A1Cresult']<6.5]
#No healty and bad social
interA=listB[listB.isin(listA)]
interA=interA[interA['A1Cresult']>0]
a=len(interA)
#Healthy and bad social
interB=listB[listB.isin(listC)]
interB=interB[interB['A1Cresult']>0]
b=len(interB)
#No healty and social
interC=listD[listD.isin(listA)]
interC=interC[interC['A1Cresult']>0]
c=len(interC)
#Healthy and social
interD=listD[listD.isin(listC)]
interD=interD[interD['A1Cresult']>0]
d=len(interD)

print(len(listB), len(listA), len(listC),len(listD))
print(a,b,c,d)

OR=(a*d)/(c*d)
RR=(a/(a+b)) / (c/(c+d))
RD=(a/(a+b)) - (c/(c+d))
print("A1CResult-->","RR: ",RR,", RD: ",RD,", OR: ",OR)
#Characteristics
listCHF=[]
for x in range (0,len(df_2)):
    if (df_2.loc[x,'Chronic_pulmonary_disease']==1):
        listCHF.append(df_2.loc[x,:])    
df_CHF = pd.DataFrame(listCHF)
print(df_CHF['age'].describe())

print("Age",mean_confidence_interval(df_CHF['age']))
print("Gender",df_CHF['gender'].astype('category').value_counts())
print("Race",df_CHF['race'].astype('category').value_counts())
print("Readmitted",df_CHF['readmitted'].astype('category').value_counts())

0.0    62774
1.0     8744
Name: Chronic_pulmonary_disease, dtype: int64
count    2866.000000
mean       93.315073
std        25.400488
min        25.000000
25%        75.000000
50%        88.000000
75%       125.000000
max       201.000000
Name: weight, dtype: float64
mean 93.31507327285415
760 287 2579 2106
84 676 203 1903
Weight--> RR:  1.1466424682395644 , RD:  0.01413505273154396 , OR:  0.41379310344827586
count    12389.000000
mean         6.357172
std          3.549122
min          1.000000
25%          1.000000
50%          8.000000
75%          9.000000
max          9.000000
Name: A1Cresult, dtype: float64
mean 6.357171684558883
8642 1560 10829 3747
1040 7602 520 3227
A1CResult--> RR:  0.8671603795417727 , RD:  -0.018435175510616006 , OR:  2.0
count    8744.000000
mean       73.315416
std        13.543032
min        10.000000
25%        60.000000
50%        70.000000
75%        80.000000
max       100.000000
Name: age, dtype: float64
Age (73.315416285452883, 73.03151389885582, 

In [28]:
print(df_2.columns)

Index(['patient_nbr', 'race', 'gender', 'age', 'weight', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'Myocardial_infarction', 'Congestive_heart_failure',
       'Peripheral_vascular_disease', 'Cerebrovascular_disease', 'Dementia',
       'Chronic_pulmonary_disease'],
      dtype='object')


In [34]:
to_del2 = ['diag_1', 'diag_2','diag_3','admission_type_id','discharge_disposition_id','admission_source_id',
           'nateglinide','chlorpropamide','acetohexamide','tolbutamide','acarbose','miglitol','troglitazone',
'tolazamide','examide','citoglipton','glyburide-metformin','glipizide-metformin','glimepiride-pioglitazone',
'metformin-rosiglitazone','metformin-pioglitazone','patient_nbr']
print (to_del2)
#Filter_selected cols
filtered_cols = [c for c in df_2.columns if (c not in to_del2) ]#and ('ENF' not in c)
df_3 = df_2[filtered_cols]
print ("df_3",df_3.shape)
print(df_3.columns)

['diag_1', 'diag_2', 'diag_3', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'nateglinide', 'chlorpropamide', 'acetohexamide', 'tolbutamide', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'patient_nbr']
df_3 (71518, 31)
Index(['race', 'gender', 'age', 'weight', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',
       'rosiglitazone', 'insulin', 'change', 'diabetesMed', 'readmitted',
       'Myocardial_infarction', 'Congestive_heart_failure',
       'Peripheral_vascular_disease', 'Cerebrovascular_disease', 'Dementia',
       'Chronic_pulmonary_disease'],


In [35]:
mid = df_3['readmitted']
df_3.drop(labels=['readmitted'], axis=1,inplace = True)
df_3.insert(0, 'readmitted', mid)
print(df_3.shape)


msk = np.random.rand(len(df_3)) < 0.8
df, df_test = df_3[msk].copy(deep = True), df_3[~msk].copy(deep = True)
df = df.reset_index()
df_test = df_test.reset_index()
print("df.shape",df.shape)
print("df_test.shape",df_test.shape)
y_train=df['readmitted']
y_test=df_test['readmitted']
print(set(y_train))
print(set(y_test))

x_train=df.iloc[:,1:50]
x_test=df_test.iloc[:,1:50]
print(set(x_train))
print(set(x_test))

(71518, 31)
df.shape (57282, 32)
df_test.shape (14236, 32)
{0, 29, 31}
{0, 29, 31}
{'Chronic_pulmonary_disease', 'time_in_hospital', 'repaglinide', 'insulin', 'Dementia', 'num_lab_procedures', 'Peripheral_vascular_disease', 'race', 'weight', 'A1Cresult', 'pioglitazone', 'glyburide', 'number_inpatient', 'Myocardial_infarction', 'glipizide', 'Cerebrovascular_disease', 'gender', 'glimepiride', 'number_outpatient', 'number_emergency', 'max_glu_serum', 'number_diagnoses', 'change', 'age', 'num_procedures', 'readmitted', 'num_medications', 'diabetesMed', 'Congestive_heart_failure', 'rosiglitazone', 'metformin'}
{'Chronic_pulmonary_disease', 'time_in_hospital', 'repaglinide', 'insulin', 'Dementia', 'num_lab_procedures', 'Peripheral_vascular_disease', 'race', 'weight', 'A1Cresult', 'pioglitazone', 'glyburide', 'number_inpatient', 'Myocardial_infarction', 'glipizide', 'Cerebrovascular_disease', 'gender', 'glimepiride', 'number_outpatient', 'number_emergency', 'max_glu_serum', 'number_diagnoses'

C:\Users\laia.subirats\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [36]:
print(df_3.columns)

Index(['readmitted', 'race', 'gender', 'age', 'weight', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',
       'rosiglitazone', 'insulin', 'change', 'diabetesMed',
       'Myocardial_infarction', 'Congestive_heart_failure',
       'Peripheral_vascular_disease', 'Cerebrovascular_disease', 'Dementia',
       'Chronic_pulmonary_disease'],
      dtype='object')


In [37]:
#Fill na
df_4 = df_3.fillna(value=np.mean(df_3,axis=0),inplace=False,axis=0).values
print ("df_4",df_4.shape)

scaler = preprocessing.MinMaxScaler().fit(df_4)
data = scaler.transform(df_4)
print ("data",data.shape)

df_4 (71518, 31)
data (71518, 31)


In [38]:
thrd = 0.8
total = 0
pca = PCA().fit(data)
reduced_data = pca.transform(data)
for pca_comps,r in enumerate(pca.explained_variance_ratio_):
    if total > thrd:
        break
    total += r
print ("Num pca_comps per >", thrd,"ratio:", pca_comps, total)
print ("Explained variance first 2 components",pca.explained_variance_ratio_[0]+pca.explained_variance_ratio_[1])
print (pca.n_components_)
print (np.sum(pca.explained_variance_ratio_[:2]))

print ("PCA+K-means:", pca_comps)
print("1st component: ", pca.components_[0])
print("2nd component: ", pca.components_[1])

Num pca_comps per > 0.8 ratio: 10 0.82108734851
Explained variance first 2 components 0.34196456945
31
0.34196456945
PCA+K-means: 10
1st component:  [ -6.54508830e-02  -2.42372736e-03   1.21043449e-02   1.10046721e-02
  -5.13202323e-04  -5.52134371e-02  -2.65832737e-02  -6.32646834e-03
  -5.00443650e-02  -1.41951471e-03  -8.65235495e-04  -3.06666218e-03
  -1.43182170e-02  -6.45024951e-03  -9.33678288e-02  -6.22236954e-02
  -4.01596583e-03  -1.44111887e-02  -3.19607472e-02  -2.62624622e-02
  -2.40877655e-02  -2.07365045e-02  -4.18428488e-02  -7.87240856e-01
  -5.92600740e-01  -3.42588728e-03  -1.39803076e-02   1.00798130e-04
  -2.97554338e-03   6.36956916e-04  -2.14084971e-02]
2nd component:  [  9.63203132e-01  -2.30873863e-02   1.32805051e-02   4.68847158e-02
  -3.10667116e-05   3.72484489e-02   3.99480058e-02  -2.90504352e-02
   1.26562268e-02   5.96367889e-03   2.41811040e-03   2.62874904e-02
   5.10095126e-02  -1.10903650e-03  -3.06061166e-02  -1.69534926e-02
   7.55281843e-04  -2.0

In [39]:
from sklearn.metrics import silhouette_samples, silhouette_score
Resultk=[0]*9
ResultC=[0]*9
for k in [2,3,4,5,6,7,8,9,10]:
    kmeans = KMeans(init='k-means++', n_clusters=k, n_init=10)
    cluster_labels = kmeans.fit_predict(reduced_data[:,:pca_comps])
    #cluster_labels=kmeans.fit(reduced_data[:,:2])
    #silhouette_avg = silhouette_score(reduced_data[:,:pca_comps], cluster_labels)
    #silhouette_avg = silhouette_score(reduced_data[:,:2], cluster_labels)
    #print("For n_clusters =", k, "The average silhouette_score is :", silhouette_avg)
    calinski_harabaz_score_avg = metrics.calinski_harabaz_score(reduced_data[:,:pca_comps], cluster_labels)
    #calinski_harabaz_score_avg = metrics.calinski_harabaz_score(reduced_data[:,:2], cluster_labels)
    print("For n_clusters =", k," the average metrics.calinski_harabaz_score is :", calinski_harabaz_score_avg)
    Resultk[k-2]=k
    ResultC[k-2]=calinski_harabaz_score_avg    
plt.plot(Resultk,ResultC,'r*-.')

For n_clusters = 2  the average metrics.calinski_harabaz_score is : 22750.4248819
For n_clusters = 3  the average metrics.calinski_harabaz_score is : 19556.6461655
For n_clusters = 4  the average metrics.calinski_harabaz_score is : 17906.5689924
For n_clusters = 5  the average metrics.calinski_harabaz_score is : 16160.5801127
For n_clusters = 6  the average metrics.calinski_harabaz_score is : 15155.6484371
For n_clusters = 7  the average metrics.calinski_harabaz_score is : 13923.4605209
For n_clusters = 8  the average metrics.calinski_harabaz_score is : 13676.7911692
For n_clusters = 9  the average metrics.calinski_harabaz_score is : 13287.4969123
For n_clusters = 10  the average metrics.calinski_harabaz_score is : 12665.4648466


In [40]:
n_clusters = 2
kmeans = KMeans(init='k-means++', n_clusters=2, n_init=10)
pca = PCA().fit(data)
cluster_labels = kmeans.fit_predict(reduced_data[:,:2])
plt.figure()
plt.plot(range(len(pca.explained_variance_ratio_)), np.cumsum(pca.explained_variance_ratio_))
plt.axvline(pca_comps, color="red")  
plt.ylim(0.0,1.1)
plt.show()

In [57]:
n_clusters=2
reduced_data = pca.transform(data)
#print ("Reduced data: ",reduced_data.shape)
kmeans = KMeans(init='k-means++', n_clusters=n_clusters, n_init=10)
kmeans.fit(reduced_data[:,:2])

# Step size of the mesh. Decrease to increase the quality of the VQ.
h = .02     # point in the mesh [x_min, m_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = reduced_data[:, 0].min()-.05, reduced_data[:, 0].max()+.05
y_min, y_max = reduced_data[:, 1].min()-.05, reduced_data[:, 1].max()+.05
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize=(10,10))
#plt.clf()
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap= plt.cm.Pastel2,#cmap=plt.cm.Paired,
           aspect='auto', origin='lower')

#plt.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=4)
print(df_4.shape)
print(data.shape)
print(reduced_data.shape)
"""
listA=df_3[df_3['readmitted'] == 0].index
plt.plot(reduced_data[listA, 0], reduced_data[listA, 1],'k.', markersize=4, c='g', label='No Readmitted')
listB=df_3[df_3['readmitted'] == 29].index
plt.plot(reduced_data[listB, 0],reduced_data[listB, 1],'k.', markersize=4, c='r',label ='<30')
listC=df_3[df_3['readmitted'] == 31].index
plt.plot(reduced_data[listC, 0],reduced_data[listC, 1],'k.', markersize=4, c='b',label ='>30')
"""
listA=df[df['change'] == 0].index
plt.plot(reduced_data[listA, 0], reduced_data[listA, 1],'k.', markersize=4, c='g', label='No disease')
listB=df[df['change'] == 1].index
plt.plot(reduced_data[listB, 0],reduced_data[listB, 1],'k.', markersize=4, c='r',label ='Disease')

#Peripheral_vascular_disease', 'Cerebrovascular_disease', 'Dementia',
#       'Chronic_pulmonary_disease', 'Myocardial_infarction',
#       'Congestive_heart_failure'

# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(centroids[0, 0], centroids[0, 1],
            marker='>', s=169, linewidths=3, 
            color='black', zorder=10)
plt.scatter(centroids[1, 0], centroids[1, 1],
            marker='H', s=169, linewidths=3, 
            color='black', zorder=10)
"""
plt.scatter(centroids[2, 0], centroids[2, 1],
            marker='>', s=169, linewidths=3, 
            color='blue', zorder=10)
plt.scatter(centroids[3, 0], centroids[3, 1],
            marker='H', s=169, linewidths=3, 
            color='blue', zorder=10)
plt.scatter(centroids[4, 0], centroids[4, 1],
            marker='>', s=169, linewidths=3, 
            color='purple', zorder=10)
plt.scatter(centroids[5, 0], centroids[5, 1],
            marker='H', s=169, linewidths=3, 
            color='purple', zorder=10)
            """
plt.title('K-means clustering on the diabetes dataset (PCA-reduced data)\n'
          'Centroids are marked with white cross')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.legend()
plt.show()

(101766, 31)
(101766, 31)
(101766, 31)


In [41]:
n_clusters=2
print(set(cluster_labels))
num_Myocardial_infarction = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    if (data[i][df_3.columns.get_loc("Myocardial_infarction")] == 0):
        num_Myocardial_infarction[cluster_labels[i]] = num_Myocardial_infarction[cluster_labels[i]] + 1
print ("cluster_sizes = ", [ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ])
print ("Myocardial infarction = ", num_Myocardial_infarction)

num_Peripheral_vascular_disease = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    if (data[i][df_3.columns.get_loc("Peripheral_vascular_disease")] == 0):
        num_Peripheral_vascular_disease[cluster_labels[i]] = num_Peripheral_vascular_disease[cluster_labels[i]] + 1
print ("cluster_sizes = ", [ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ])
print ("num_no_Peripheral_vascular_disease = ", num_Peripheral_vascular_disease)

num_Cerebrovascular_disease = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    if (data[i][df_3.columns.get_loc("Cerebrovascular_disease")] == 0):
        num_Cerebrovascular_disease[cluster_labels[i]] = num_Cerebrovascular_disease[cluster_labels[i]] + 1
print ("cluster_sizes = ", [ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ])
print ("num_no_Cerebrovascular_disease = ", num_Cerebrovascular_disease)

num_Dementia = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    if (data[i][df_3.columns.get_loc("Dementia")] == 0):
        num_Dementia[cluster_labels[i]] = num_Dementia[cluster_labels[i]] + 1
print ("cluster_sizes = ", [ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ])
print ("num_no_Dementia = ", num_Dementia)

num_Chronic_pulmonary_disease = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    if (data[i][df_3.columns.get_loc("Chronic_pulmonary_disease")] == 0):
        num_Chronic_pulmonary_disease[cluster_labels[i]] = num_Chronic_pulmonary_disease[cluster_labels[i]] + 1
print ("cluster_sizes = ", [ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ])
print ("num_no_Chronic_pulmonary_disease = ", num_Chronic_pulmonary_disease)

num_Congestive_heart_failure = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    if (data[i][df_3.columns.get_loc("Congestive_heart_failure")] == 0):
        num_Congestive_heart_failure[cluster_labels[i]] = num_Congestive_heart_failure[cluster_labels[i]] + 1
print ("cluster_sizes = ", [ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ])
print ("num_no_Congestive_heart_failure = ", num_Congestive_heart_failure)

num_readmitted = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
    if (data[i][df_3.columns.get_loc("readmitted")] == 0):
        num_readmitted[cluster_labels[i]] = num_readmitted[cluster_labels[i]] + 1
print ("cluster_sizes = ", [ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ])
print ("num_no_readmitted = ", num_readmitted)

num_age = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
        num_age[cluster_labels[i]] = num_age[cluster_labels[i]] + df_4[i][df_3.columns.get_loc("age")]
mida_cluster=[ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ]
print ("average age = ", list(map(truediv,num_age,mida_cluster)))

num_age = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
        num_age[cluster_labels[i]] = num_age[cluster_labels[i]] + df_4[i][df_3.columns.get_loc("weight")]
mida_cluster=[ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ]
print ("average weight = ", list(map(truediv,num_age,mida_cluster)))

num_age = [0] * n_clusters
for i in range(len(reduced_data[:,:2])):
        num_age[cluster_labels[i]] = num_age[cluster_labels[i]] + df_4[i][df_3.columns.get_loc("A1Cresult")]
mida_cluster=[ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ]
print ("average A1Cresult = ", list(map(truediv,num_age,mida_cluster)))

{0, 1}
cluster_sizes =  [39505, 32013]
Myocardial infarction =  [37538, 30368]
cluster_sizes =  [39505, 32013]
num_no_Peripheral_vascular_disease =  [38760, 31410]
cluster_sizes =  [39505, 32013]
num_no_Cerebrovascular_disease =  [37325, 30128]
cluster_sizes =  [39505, 32013]
num_no_Dementia =  [39400, 31952]
cluster_sizes =  [39505, 32013]
num_no_Chronic_pulmonary_disease =  [34937, 27837]
cluster_sizes =  [39505, 32013]
num_no_Congestive_heart_failure =  [33515, 27012]
cluster_sizes =  [39505, 32013]
num_no_readmitted =  [30416, 23935]
average age =  [71.356537147196562, 70.261456283384874]
average weight =  [93.271931631564826, 93.368311349156528]
average A1Cresult =  [-0.041108720415137322, 0.66391778340049357]


In [59]:
#P-values of attributes of each cluster
import scipy.stats as stats
#true_mu = 0
listaA=df_3.columns.tolist()
cluster1=df_4[kmeans.labels_==0]
cluster2=df_4[kmeans.labels_==1]
cluster_labels=kmeans.labels_
#print(cluster1.shape)
#print(cluster2.shape)
contador=0
for z in range(0,len(df_3.columns.tolist())):
    two_sample = stats.ttest_ind(cluster1[:,z],cluster2[:,z])
    if(two_sample[1]<0.05):contador=contador+1

w, h = 3, contador
Matrix = [[0.0 for x in range(w)] for y in range(h)] 
i=0
for z in range(0,len(df_3.columns.tolist())):
    #print(df_2.loc[1,z])
    #print(np.std(cluster1[:,z]))
    #print(df_2.values[:,z])
    two_sample = stats.ttest_ind(cluster1[:,z],cluster2[:,z])
    #two_sample = stats.chisquare(cluster1[:,8],cluster2[:,8])
    if(two_sample[1]<0.05): 
        #print(i)
        Matrix[i][0]=listaA[z]
        Matrix[i][1]='{0:.400f}'.format(two_sample[1])
        Matrix[i][2]=two_sample[1]
        i=i+1
arr = np.array(Matrix)
arr = arr[arr[:,1].argsort()]
#print(arr)

for z in range(0,h):
    if (0<1):
        num_hipertensos = [0] * n_clusters
        for i in range(len(data[:,:2])):
            num_hipertensos[cluster_labels[i]] = num_hipertensos[cluster_labels[i]] + df_4[i][df_3.columns.get_loc(arr[z,0])]
        mida_cluster=[ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ]
        print ((arr[z,0]),": ", list(map(truediv,num_hipertensos,mida_cluster)),", p-value: ",arr[z,2])
    else:
        num_hipertensos = [0] * n_clusters
        for i in range(len(data[:,:2])):
            #print("z",z)
            #print("i",i)
            if (data[i][df_3.columns.get_loc(arr[z,0])] == 1):
                num_hipertensos[cluster_labels[i]] = num_hipertensos[cluster_labels[i]] + 1
        mida_cluster=[ len([1 for label in cluster_labels if label == i]) for i in range(n_clusters) ]
        percentage=list(map(truediv,num_hipertensos,mida_cluster))
        percentage[0]=round(percentage[0],2)
        percentage[1]=round(percentage[1],2)
        print ((arr[z,0]),": ", num_hipertensos,", percentage: ",percentage, "p-value: ",arr[z,2])

diabetesMed :  [1.0, 0.57258697835814076] , p-value:  0.0
change :  [1.0, 0.0] , p-value:  0.0
rosiglitazone :  [0.11193125013294761, 0.018628435759291389] , p-value:  0.0
num_medications :  [18.187041330752376, 14.162870970687608] , p-value:  0.0
pioglitazone :  [0.12590670268660525, 0.023541229111496666] , p-value:  0.0
glyburide :  [0.14477462721490714, 0.054132042735823213] , p-value:  0.0
glipizide :  [0.17574610197613325, 0.064176787507990135] , p-value:  0.0
metformin :  [0.32135032226500182, 0.077125376677928958] , p-value:  0.0
glimepiride :  [0.078598625853523646, 0.022664596840471191] , p-value:  1.90455413404e-272
time_in_hospital :  [4.7476973474293249, 4.0940188110674827] , p-value:  2.51073752735e-267
A1Cresult :  [0.60211439875773753, -0.066185736462423525] , p-value:  3.84976743072e-255
repaglinide :  [0.02454744634234541, 0.0065747420326910787] , p-value:  1.91641224807e-93
num_lab_procedures :  [44.45365978175321, 41.929686786594829] , p-value:  1.02804773373e-92
num